In [1]:
import simf
from scipy import sparse
import numpy as np

Generate some random streams data.

Define two relation matrices: `R1` and `R2`.
Relation `R1` describes the relation between objects of type `o1` (1000 objects) and `o2` (500 objects).
Relation `R2` described the relation between objects of type `o1` (1000 objects) and `o3` (300 objects).

Generate some random patterns that are shared by the two relations.

In [2]:
error = 0.1

A = np.random.rand(1000, 3*2)  # for objects of type o1
B = np.random.rand(500, 3*2)  # for objects of type o2
R1 = A.dot(B.T) + error * np.random.rand(1000, 500)

C = np.random.rand(300, 3*2)  # for objects of type o3
R2 = A.dot(C.T) + error * np.random.rand(1000, 300)

# select a sparse random subset
R1 = np.multiply(R1, sparse.random(1000, 500).todense() > 0)
R2 = np.multiply(R2, sparse.random(1000, 300).todense() > 0)

Each object type is mapped to a latent factor space, with ranks as defined:

In [3]:
o1 = simf.ObjectType('O1', 10)
o2 = simf.ObjectType('O2', 5)
o3 = simf.ObjectType('O3', 3)

Define two relations `r1` and `r2` that share a common object type `o1`.

In [4]:
r1 = simf.Relation(o1, o2, R1, weight=1)
r2 = simf.Relation(o1, o3, R2, weight=1)

The data is thus composed of two streams, one stream for each relation, `R1` and `R2`.

In [5]:
data = [r1, r2]

Fit the model to the data.

In [6]:
model = simf.SIMF()
model.fit(data)

Report the training error. Two values are reported for each relation (RMSE, MAE).

In [7]:
error = model.get_train_error()
error

{<simf.factorization_system.relation.Relation at 0x109b15668>: (0.21366836977767834,
  0.16847349732193237),
 <simf.factorization_system.relation.Relation at 0x109b15438>: (0.23998017531396734,
  0.19011481475102557)}

Make some predictions for stream `r1`.

Prediction relation `r1` between first object of type `o1` (`o1=0`) and first object of type `o2` (`o2=0`):

In [8]:
model.predict(r1, 0, 0)

2.124952184860879

Prediction relation `r1` between first object of type `o1` (`o1=0`) and second object of type `o2` (`o2=1`):

In [9]:
model.predict(r1, 0, 1)

1.7639677474440665

You can obtain the same predictions by putting the object pairs in a list, and then calling the `predict_stream` function.

In [10]:
stream_to_predict = [(0, 0, '?'), (0, 1, '?')]
model.predict_stream(r1, stream_to_predict, verbose=True)

array([ 2.12495218,  1.76396775])